#1-1

MNIST 데이터셋을 사용하여 간단한 GAN을 구현한 코드입니다.

코드를 실행시키고, 주석을 달아주세요.

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image


In [5]:
num_epochs = 10
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
noise_size = 100
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024
dir_name = "GAN_results"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if not os.path.exists(dir_name):
    os.makedirs(dir_name)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

MNIST_dataset = datasets.MNIST(root='../../data/',
                               train=True,
                               transform=transform,
                               download=True)

data_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_size, hidden_size3),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size3, hidden_size2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size2, hidden_size1),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, hidden_size3),
            nn.ReLU(),
            nn.Linear(hidden_size3, img_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

discriminator = Discriminator().to(device)
generator = Generator().to(device)

criterion = nn.BCELoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)


In [7]:
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        real_images = images.reshape(batch_size, -1).to(device)

        g_optimizer.zero_grad()

        z = torch.randn(batch_size, noise_size).to(device)
        fake_images = generator(z)

        g_loss = criterion(discriminator(fake_images), real_labels)
        g_loss.backward()
        g_optimizer.step()

        d_optimizer.zero_grad()

        z = torch.randn(batch_size, noise_size).to(device)
        fake_images = generator(z)

        real_loss = criterion(discriminator(real_images), real_labels)
        fake_loss = criterion(discriminator(fake_images.detach()), fake_labels)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        d_optimizer.step()

        if (i + 1) % 150 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    save_image(fake_images.view(batch_size, 1, 28, 28),
               os.path.join(dir_name, f'GAN_fake_image_{epoch + 1}.png'))

    d_performance = discriminator(real_images).mean().item()
    g_performance = discriminator(fake_images).mean().item()
    print(f"---------Epoch [{epoch + 1}/{num_epochs}] : D Performance: {d_performance:.2f}, G Performance: {g_performance:.2f}")

Epoch [1/10], Step [150/600], D Loss: 0.0258, G Loss: 3.6775
Epoch [1/10], Step [300/600], D Loss: 0.0450, G Loss: 12.9928
Epoch [1/10], Step [450/600], D Loss: 0.0083, G Loss: 6.9186
Epoch [1/10], Step [600/600], D Loss: 0.0080, G Loss: 11.9610
---------Epoch [1/10] : D Performance: 1.00, G Performance: 0.00
Epoch [2/10], Step [150/600], D Loss: 0.0289, G Loss: 12.0795
Epoch [2/10], Step [300/600], D Loss: 0.0258, G Loss: 12.3370
Epoch [2/10], Step [450/600], D Loss: 0.0072, G Loss: 6.9211
Epoch [2/10], Step [600/600], D Loss: 0.0065, G Loss: 10.8857
---------Epoch [2/10] : D Performance: 0.99, G Performance: 0.00
Epoch [3/10], Step [150/600], D Loss: 0.0107, G Loss: 12.2961
Epoch [3/10], Step [300/600], D Loss: 0.1075, G Loss: 17.3145
Epoch [3/10], Step [450/600], D Loss: 0.4564, G Loss: 10.5671
Epoch [3/10], Step [600/600], D Loss: 0.1781, G Loss: 13.4794
---------Epoch [3/10] : D Performance: 0.89, G Performance: 0.01
Epoch [4/10], Step [150/600], D Loss: 0.3307, G Loss: 14.3609
Ep

#1-2

아래 마크다운으로 GAN_fake_image_1.png와 GAN_fake_image_100.png를 함께 첨부해주세요.

![](https://drive.google.com/uc?export=view&id=1NpBiXBbD4o-GtBlhT8LuDzQ5_DRg1aDV)



![](https://drive.google.com/uc?export=view&id=1huNza8yAc2hp2WpCRSdpKHQ5aRfLjCF9)

In [ ]:
#실행이 너무 느려서 10번밖에 돌리지 못했습니다 ㅠㅠ